<h1> 1. 데이터 불러오기
    
<h3> kaggle에서 iris dataset을 다운받는다

In [3]:
import pandas as pd
import os  # 경로를 불러오는 패키지

In [4]:
os.chdir(r'C:\Users\hjb38\Documents\데이터 분석 과정\data')
# 앞에 r 붙여줘야함

In [5]:
iris = pd.read_csv("IRIS.csv")

In [6]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [7]:
iris['species'].value_counts()  # 꽃의 종류

Iris-virginica     50
Iris-setosa        50
Iris-versicolor    50
Name: species, dtype: int64

In [8]:
iris['id'] = range(len(iris))  # 각각의 객체를 구별하기 위해 id 라는 컬럼을 새로 추가
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species,id
0,5.1,3.5,1.4,0.2,Iris-setosa,0
1,4.9,3.0,1.4,0.2,Iris-setosa,1
2,4.7,3.2,1.3,0.2,Iris-setosa,2
3,4.6,3.1,1.5,0.2,Iris-setosa,3
4,5.0,3.6,1.4,0.2,Iris-setosa,4


In [9]:
iris = iris[['id', 'sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']]
# id 가 맨 앞으로 오게 순서 변경
iris.head()

,id,sepal_length,sepal_width,petal_length,petal_width,species
0,0,5.1,3.5,1.4,0.2,Iris-setosa
1,1,4.9,3.0,1.4,0.2,Iris-setosa
2,2,4.7,3.2,1.3,0.2,Iris-setosa
3,3,4.6,3.1,1.5,0.2,Iris-setosa
4,4,5.0,3.6,1.4,0.2,Iris-setosa


<h1> 2. KNN 실습 1 (범주형 데이터 - 분류)
    
<h3> 특징
<h4>  : 데이터가 많으면 느리다.
<h3> 파라미터
<h4>  n_neighbors : 가장 가까운 몇 개를 볼건지

1) Train(학습) 데이터 추출

In [10]:
train = iris.sample(100, replace = False, random_state = 2020)
# 전체 150개 데이터 중 100개의 데이터를 샘플로하여 비복원추출
# 나머지 50개는 테스트 데이터로 쓸 것임

train.head()

,id,sepal_length,sepal_width,petal_length,petal_width,species
104,104,6.5,3.0,5.8,2.2,Iris-virginica
8,8,4.4,2.9,1.4,0.2,Iris-setosa
61,61,5.9,3.0,4.2,1.5,Iris-versicolor
54,54,6.5,2.8,4.6,1.5,Iris-versicolor
78,78,6.0,2.9,4.5,1.5,Iris-versicolor


In [11]:
# index의 순서가 뒤직박죽일 때는 reset으로 컬럼으로 빼준 뒤 없앤다.
train = train.reset_index().drop(['index'], axis=1)
train.head()

,id,sepal_length,sepal_width,petal_length,petal_width,species
0,104,6.5,3.0,5.8,2.2,Iris-virginica
1,8,4.4,2.9,1.4,0.2,Iris-setosa
2,61,5.9,3.0,4.2,1.5,Iris-versicolor
3,54,6.5,2.8,4.6,1.5,Iris-versicolor
4,78,6.0,2.9,4.5,1.5,Iris-versicolor


2) Test(검증) 데이터 추출

In [12]:
~iris['id'].isin(train['id'])
# iris 의 id가 train 의id에 포함되는가? True면 train 용 데이터 False면 test 용 데이터
# test 용 데이터인 False 를 사용해야하기 때문에 ~ 를 이용해 test 용 데이터를 True로 바꿔준다.

0      False
1       True
2      False
3       True
4      False
       ...  
145    False
146    False
147    False
148    False
149    False
Name: id, Length: 150, dtype: bool

In [13]:
test = iris.loc[~iris['id'].isin(train['id'])]
test.head()

,id,sepal_length,sepal_width,petal_length,petal_width,species
1,1,4.9,3.0,1.4,0.2,Iris-setosa
3,3,4.6,3.1,1.5,0.2,Iris-setosa
6,6,4.6,3.4,1.4,0.3,Iris-setosa
9,9,4.9,3.1,1.5,0.1,Iris-setosa
20,20,5.4,3.4,1.7,0.2,Iris-setosa


In [14]:
# index가 또 뒤죽박죽 돼서 다시 reset
test.reset_index().drop(['index'], axis=1)
test.head()

,id,sepal_length,sepal_width,petal_length,petal_width,species
1,1,4.9,3.0,1.4,0.2,Iris-setosa
3,3,4.6,3.1,1.5,0.2,Iris-setosa
6,6,4.6,3.4,1.4,0.3,Iris-setosa
9,9,4.9,3.1,1.5,0.1,Iris-setosa
20,20,5.4,3.4,1.7,0.2,Iris-setosa


3) train 데이터를 학습시키기 위해 패키지를 불러옴

In [15]:
from sklearn.neighbors import KNeighborsClassifier

4) 모델 정의하기

In [16]:
knn = KNeighborsClassifier(n_neighbors = 3)
# n_neighbors를 곧 K라고 생각하면 됨, 임의로 3이라고 지정

5) 모델 학습시키기

In [17]:
knn.fit(train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], train['species'])
# 인풋변수들(sepal_length ~ )petal_width 및 타겟변수(species)를 지정한다.

KNeighborsClassifier(n_neighbors=3)

6) 학습된 모델을 바탕으로 test 데이터에 예측해보기

In [18]:
predictions = knn.predict(test[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])

In [19]:
test['pred'] = predictions  # pred라는 컬럼을 추가해 실제 값인 species 와 비교한다
test.head()

<ipython-input-19-2103b32d8e41>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = predictions  # pred라는 컬럼을 추가해 실제 값인 species 와 비교한다


,id,sepal_length,sepal_width,petal_length,petal_width,species,pred
1,1,4.9,3.0,1.4,0.2,Iris-setosa,Iris-setosa
3,3,4.6,3.1,1.5,0.2,Iris-setosa,Iris-setosa
6,6,4.6,3.4,1.4,0.3,Iris-setosa,Iris-setosa
9,9,4.9,3.1,1.5,0.1,Iris-setosa,Iris-setosa
20,20,5.4,3.4,1.7,0.2,Iris-setosa,Iris-setosa


In [20]:
# 실제값인 species와  예측값인 pred가 얼마나 잘 맞는지 비교한다.
(test['species'] == test['pred']).mean()

# k가 3일때 98% 의 정확도가 나옴

0.98

<h1> 3. 최적의 K 찾기

In [21]:
# 위의 코드를 그대로 불러온 뒤 n_neighbors =  의 값을 반복문으로 바꿔준다.

for k in range(1,30):

    knn = KNeighborsClassifier(n_neighbors = k) 

    knn.fit(train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], train['species'])

    predictions = knn.predict(test[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])

    test['pred'] = predictions  

    print((test['species'] == test['pred']).mean())

# 만약 가장 높은 확률을 가진 k가 여러개라면 그 중 가장 작은 k를 선택해야 데이터 처리 시간을 단축할 수 있다.

0.94
0.94
0.98
0.98
0.98
0.96
0.98
0.98
0.98
0.98
0.98
0.98
0.98
0.98
0.98
0.98
0.98
0.98
0.98
0.96
0.96
0.96
0.96
0.96
0.96
0.96
0.96
0.96
0.96


<ipython-input-21-97e238cdbfde>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = predictions
<ipython-input-21-97e238cdbfde>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = predictions
<ipython-input-21-97e238cdbfde>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

k가 3일 때 98%로 가장 최적이다.

<h3> 사실 위의 방법은 고전적인 방법이고

<h3> 요즘은 Cross validation 방법을 쓴다. [21_의사결정나무] 참고 

In [31]:
from sklearn.model_selection import cross_val_score
import numpy as np

In [37]:
for k in range(1,30):

    knn = KNeighborsClassifier(n_neighbors = k)
    print(np.mean(cross_val_score(knn, iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], iris['species'], cv=3)))

0.96
0.9466666666666667
0.9733333333333333
0.9866666666666667
0.98
0.9733333333333333
0.9733333333333333
0.9666666666666667
0.9733333333333333
0.9666666666666667
0.9666666666666667
0.9666666666666667
0.96
0.9533333333333333
0.9533333333333333
0.96
0.96
0.9533333333333333
0.9533333333333333
0.9466666666666667
0.9533333333333333
0.9466666666666667
0.94
0.9333333333333332
0.94
0.9533333333333333
0.9533333333333333
0.9533333333333333
0.9533333333333333


<h3> Cross validation 방법을 사용했을 때 최적의 K는 4다.

<h1> 4. KNN 실습 2 (연속형 데이터 - 회귀)



<h3> sepal_length, sepal_width, petal_length 로 petal_width 예측하기

In [22]:
# species는 필요없으니 지움
del train['species']
del test['species']

In [23]:
from sklearn.neighbors import KNeighborsRegressor

In [24]:
knn = KNeighborsRegressor(n_neighbors = 3)

In [25]:
knn.fit(train[['sepal_length', 'sepal_width', 'petal_length']], train['petal_width'])
# 인풋변수와 타겟변수 지정

KNeighborsRegressor(n_neighbors=3)

In [26]:
# predict 함수로 예측하기
knn.predict(test[['sepal_length', 'sepal_width', 'petal_length']])
# 그럼 연속형 변수인 숫자로 결과가 나옴


array([0.16666667, 0.2       , 0.2       , 0.13333333, 0.36666667,
       0.23333333, 0.23333333, 0.2       , 0.3       , 0.16666667,
       0.16666667, 0.23333333, 1.53333333, 1.33333333, 1.26666667,
       1.7       , 1.4       , 1.33333333, 1.26666667, 1.43333333,
       1.2       , 1.33333333, 1.7       , 1.16666667, 1.2       ,
       1.26666667, 1.86666667, 1.3       , 1.53333333, 1.26666667,
       1.        , 2.33333333, 2.16666667, 2.        , 2.        ,
       2.13333333, 1.86666667, 2.        , 2.36666667, 2.36666667,
       1.8       , 2.16666667, 2.26666667, 1.9       , 2.        ,
       2.13333333, 1.8       , 2.36666667, 2.2       , 1.8       ])

In [27]:
test['pred'] = knn.predict(test[['sepal_length', 'sepal_width', 'petal_length']])
# 위 데이터를 test['pred'] 컬럼에 저장
test.head()
# 연속형 데이터이기 때문에 분류형과는 다르게 완전히 똑같을 가능성이 거의 없다.
# 따라서 실제값과 예측값과의 거리를 비교한다.

<ipython-input-27-367933f1df15>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = knn.predict(test[['sepal_length', 'sepal_width', 'petal_length']])


,id,sepal_length,sepal_width,petal_length,petal_width,pred
1,1,4.9,3.0,1.4,0.2,0.166667
3,3,4.6,3.1,1.5,0.2,0.200000
6,6,4.6,3.4,1.4,0.3,0.200000
9,9,4.9,3.1,1.5,0.1,0.133333
20,20,5.4,3.4,1.7,0.2,0.366667


mae : 회기문제에서 모델의 성능을 평가하는 방법

In [28]:
# mae 계산식 : 절대값(실제값 - 예측값)의 평균
# 예시
(abs(0.2-0.266667) + abs(0.2-0.233333) + abs(0.2-0.233333) + abs(0.4-0.3) + abs(0.2-0.266667))/5

# 결과가 작을 수록 예측을 잘했다는 뜻.

0.06

In [29]:
abs(test['petal_width']-test['pred']).mean()

0.1673333333333333

<h1> 5. 최적의 K 찾기

In [30]:
for k in range(1,30):

    knn = KNeighborsRegressor(n_neighbors = k) 

    knn.fit(train[['sepal_length', 'sepal_width', 'petal_length']], train['petal_width'])

    test['pred'] = knn.predict(test[['sepal_length', 'sepal_width', 'petal_length']])

    print(abs(test['petal_width']-test['pred']).mean())

0.196
0.16699999999999995
0.1673333333333333
0.1655
0.16839999999999994
0.1619999999999999
0.16342857142857137
0.16125
0.16155555555555556
0.16260000000000002
0.16254545454545455
0.1635
0.1636923076923077
0.1695714285714286
0.1738666666666667
0.17587499999999995
0.18282352941176455
0.18955555555555553
0.1913684210526316
0.19329999999999994
0.19780952380952382
0.20045454545454547
0.20330434782608697
0.2045
0.2084800000000001
0.2123076923076923
0.21599999999999997
0.21878571428571422
0.22365517241379315


<ipython-input-30-b26076a7cf73>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = knn.predict(test[['sepal_length', 'sepal_width', 'petal_length']])
<ipython-input-30-b26076a7cf73>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = knn.predict(test[['sepal_length', 'sepal_width', 'petal_length']])
<ipython-input-30-b26076a7cf73>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

k가 8일 때 가장 mae 가 낮다